In [1]:
from jinja2 import Template
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import re

d:\Cours\Master\NLP in industry\publication-date-extractor\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt_template = Template(
    """
    You are an expert in structured data extraction.
    Given a document in French, extract and output only the **publication date** of the document in the format DD/MM/YYYY. Do not include any additional text or context — just the date.

    Document: "{{ document }}"
    Publication date:
    """
)

In [3]:
!pip install -U bitsandbytes


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
unsloth_checkpoint = "unsloth/Llama-3.2-3B"
model = AutoModelForCausalLM.from_pretrained(unsloth_checkpoint,
                                             load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(unsloth_checkpoint)

def llm_complete(prompt, max_tokens=2048, device = 'cuda', temperature=0.5):
    # Fill it
    # model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs.to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, temperature = temperature, pad_token_id=tokenizer.eos_token_id)
    outputs_ans_only = outputs[:,len(inputs['input_ids'][0]):]
    answer_only = tokenizer.batch_decode(outputs_ans_only, skip_special_tokens=True)
    return answer_only

d:\Cours\Master\NLP in industry\publication-date-extractor\env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alex\.cache\huggingface\hub\models--unsloth--Llama-3.2-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in 

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#llm_complete(prompt, max_tokens = 30)

TODO:
- use regex to generate text chunks, then join them into one long context


In [8]:
df = pd.read_pickle('df_with_regex_chunks.pkl')

In [9]:
def predict_date(document, temperature=0.5):
    input = {'document': document}
    prompt = prompt_template.render(**input)
    output = llm_complete(prompt, max_tokens = 10, temperature=temperature)
    date = re.findall(r"\d{2}/\d{2}/\d{4}\b", str(output))
    if date: return date[0]
    else: return str(output)

In [10]:
test = df.sample(10)
print(test['Gold published date'])
test['prediction'] = test.apply(lambda x: predict_date(str(x['regex_chunks'])+x['doc_id']), axis=1)
print(test['prediction'])

175    31/01/2023
248    12/12/2022
3      26/01/2023
231    20/03/2024
396    30/09/2020
208    03/10/2022
70     06/02/2023
41     03/11/2022
360          None
417    25/03/2013
Name: Gold published date, dtype: object


d:\Cours\Master\NLP in industry\publication-date-extractor\env\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


175                                     08/02/2023
248                                     12/12/2022
3                                       26/01/2023
231                                     06/03/2024
396    [' 30 septembre 2020\n    Expected output']
208                              [' 2022-06-27\n']
70                 [' 2015-12-17\n\n    Document']
41                                      04/09/2019
360                                     26/11/2020
417                                     25/03/2013
Name: prediction, dtype: object


In [11]:
df['prediction'] = df.apply(lambda x: predict_date(str(x['regex_chunks'])+x['doc_id']), axis=1)

In [12]:
df[['Gold published date','prediction']]

,Gold published date,prediction
0,16/01/2023,[' * 16 January 2023\n Expected']
1,25/01/2023,25/01/2023
2,02/02/2023,02/02/2023
3,26/01/2023,26/01/2023
4,16/01/2023,16/01/2023
...,...,...
495,02/04/2024,24/01/2024
496,09/01/2024,[' - 2024-01-10\n ']
497,22/11/2022,22/11/2022
498,21/12/2023,15/12/2023


In [15]:
df.to_pickle('huggingface_llama.pkl')